In [2]:
import numpy as np

from pyspark.sql import SparkSession
from pyspark.sql.types import *

from pyspark import SparkConf
conf = SparkConf()
conf.set('spark.executor.memory', '2g')
spark = SparkSession.builder.appName('kmeans').getOrCreate()
# Load Data
data = spark.read.csv("dataset/US_Accidents_Dec20_Updated.csv", header=True, sep=",")

In [3]:
df = data[["ID","Start_Lat","Start_Lng","Temperature(F)", "Humidity(%)", "Pressure(in)", "Visibility(mi)",]]
df.show()
df.count()

+----+------------------+-------------------+--------------+-----------+------------+--------------+
|  ID|         Start_Lat|          Start_Lng|Temperature(F)|Humidity(%)|Pressure(in)|Visibility(mi)|
+----+------------------+-------------------+--------------+-----------+------------+--------------+
| A-1|         34.808868| -82.26915699999999|          76.0|       52.0|       28.91|          10.0|
| A-2|          35.09008|          -80.74556|          76.0|       62.0|        29.3|          10.0|
| A-3|          37.14573|        -121.985052|          51.0|       80.0|       30.17|          10.0|
| A-4|          39.11039|        -119.773781|          53.6|       16.0|       30.16|          10.0|
| A-5|26.102942000000002|         -80.265091|          84.2|       84.0|       29.92|          10.0|
| A-6|35.348240000000004| -80.84722099999999|          73.4|       33.0|       30.17|          10.0|
| A-7|          39.52397|           -107.777|          28.0|       88.0|       24.67|      

2906610

In [4]:
df = df.withColumn("Start_Lat",df['Start_Lat'].cast(DoubleType()))\
    .withColumn("Start_Lng",df['Start_Lng'].cast(DoubleType()))\
    .withColumn("Temperature(F)",df['Temperature(F)'].cast(DoubleType()))\
    .withColumn("Humidity(%)",df['Humidity(%)'].cast(DoubleType()))\
    .withColumn("Pressure(in)",df['Pressure(in)'].cast(DoubleType()))\
    .withColumn("Visibility(mi)",df['Visibility(mi)'].cast(DoubleType()))
df.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Start_Lat: double (nullable = true)
 |-- Start_Lng: double (nullable = true)
 |-- Temperature(F): double (nullable = true)
 |-- Humidity(%): double (nullable = true)
 |-- Pressure(in): double (nullable = true)
 |-- Visibility(mi): double (nullable = true)



In [5]:
df = df.dropna()
df.count()

2816332

In [6]:
import numpy as np
df.show()

+----+------------------+-------------------+--------------+-----------+------------+--------------+
|  ID|         Start_Lat|          Start_Lng|Temperature(F)|Humidity(%)|Pressure(in)|Visibility(mi)|
+----+------------------+-------------------+--------------+-----------+------------+--------------+
| A-1|         34.808868| -82.26915699999999|          76.0|       52.0|       28.91|          10.0|
| A-2|          35.09008|          -80.74556|          76.0|       62.0|        29.3|          10.0|
| A-3|          37.14573|        -121.985052|          51.0|       80.0|       30.17|          10.0|
| A-4|          39.11039|        -119.773781|          53.6|       16.0|       30.16|          10.0|
| A-5|26.102942000000002|         -80.265091|          84.2|       84.0|       29.92|          10.0|
| A-6|35.348240000000004| -80.84722099999999|          73.4|       33.0|       30.17|          10.0|
| A-7|          39.52397|           -107.777|          28.0|       88.0|       24.67|      